In [21]:
import twstock
from twstock import stock
import sqlite3
import pandas as pd
from datetime import datetime
import os

In [22]:
strdate = datetime.now().strftime("%Y%m%d")

In [23]:
def save_stock(sid:str,strdate=None):
    stock_data = stock.Stock(sid)
    
    if strdate == None:
        strdate = datetime.now().strftime("%Y%m%d")        
    else:
        year = int(strdate[0:4]) ; month = int(strdate[4:6])
        stock_data.fetch(year=year,month=month)

    try:
        os.mkdir(strdate)
    except:        
        pass

    conn = sqlite3.connect('%s/%s.db' % (strdate,sid),detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()
 
    # Create table
    cursor.execute('''CREATE TABLE IF NOT EXISTS stocks
        (date timestamp, capacity integer, turnover text, open real, high real, 
        low real, close real, change real, transactions integer)''')

    # Insert a row of data
    for data in stock_data.data:
        cursor.execute("INSERT INTO stocks VALUES (?,?,?,?,?,?,?,?,?)",data)    

    # Save (commit) the changes
    conn.commit()
 
    # We can also close the connection if we are done with it.
    # Just be sure any changes have been committed or they will be lost.
    conn.close()

In [24]:
def read_stock(sid:str,strdate=None):
    if strdate == None:
        strdate = datetime.now().strftime("%Y%m%d")

    conn = sqlite3.connect('%s/%s.db' % (strdate,sid),detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()

    # Read table
    sqlite_data = cursor.execute('SELECT * FROM stocks').fetchall()
    
    data_pd = pd.DataFrame(sqlite_data,columns=['date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction'])
    
    return data_pd

# 收集所有的股號

In [25]:
def get_stockids():
    sids = []
    twse = twstock.twse
    for sid in twse.keys():
        if twse[sid].type == '股票':
            sids.append(sid)
    return sids

# 儲存最近日期之股市資料至sqlite

In [26]:
sids = get_stockids()
for sid in sids:
    print('Downloading ...%5s'%(sid))
    try:
        read_stock(sid)
    except:
        try:
            save_stock(sid)
        except:
            print(twse[sid].name,sid,' Calculate failed')

# 儲存指定日期之股市資料至sqlite

In [ ]:
strdate='20180801'
sids = get_stockids()
for sid in sids:
    print('Downloading ...%5s'%(sid))
    try:
        read_stock(sid,strdate)
    except:
        try:
            save_stock(sid,strdate)
        except:
            print(twse[sid].name,sid,' Calculate failed')